In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import yfinance as yf
import time
from pathlib import Path
import os
import openpyxl

In [18]:
# Get the current working directory
current_dir = os.getcwd()

# Resolve the parent directories
project_general_path = Path(current_dir).resolve()
print(project_general_path)

C:\Users\Adam Krupa\OneDrive\Pulpit\Investing\earnings-report-analysis


In [7]:
project_general_path = Path(__file__).resolve().parent.parent
print(project_general_path)


NameError: name '__file__' is not defined

In [42]:
def fetch_tickers():
    # URL of the website containing the S&P 500 tickers
    url = 'https://www.slickcharts.com/sp500'

    # Fetch the page content with headers to avoid being blocked
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table', {'class': 'table table-hover table-borderless table-sm'})
        tickers = []
        
        # Extract tickers from the table
        if table:
            for row in table.find('tbody').find_all('tr'):
                columns = row.find_all('td')
                ticker = columns[2].text.strip()  # The third column contains the ticker symbol
                tickers.append(ticker)
        else:
            print("Table not found in the page")
    else:
        print("Failed to fetch S&P 500 tickers")
        tickers = []

    return tickers
    


In [43]:
fetch_tickers()

['AAPL',
 'NVDA',
 'MSFT',
 'AMZN',
 'META',
 'TSLA',
 'GOOGL',
 'AVGO',
 'GOOG',
 'BRK.B',
 'JPM',
 'LLY',
 'V',
 'XOM',
 'UNH',
 'MA',
 'COST',
 'PG',
 'WMT',
 'HD',
 'NFLX',
 'JNJ',
 'CRM',
 'ABBV',
 'BAC',
 'ORCL',
 'MRK',
 'KO',
 'CVX',
 'CSCO',
 'WFC',
 'ACN',
 'NOW',
 'MCD',
 'PEP',
 'IBM',
 'AMD',
 'DIS',
 'LIN',
 'TMO',
 'ABT',
 'ADBE',
 'PM',
 'ISRG',
 'GE',
 'GS',
 'INTU',
 'CAT',
 'QCOM',
 'TXN',
 'VZ',
 'BKNG',
 'AXP',
 'PLTR',
 'T',
 'SPGI',
 'RTX',
 'MS',
 'BLK',
 'PFE',
 'NEE',
 'HON',
 'DHR',
 'CMCSA',
 'AMGN',
 'PGR',
 'LOW',
 'AMAT',
 'UNP',
 'TJX',
 'ETN',
 'BA',
 'BSX',
 'C',
 'UBER',
 'SYK',
 'COP',
 'BX',
 'PANW',
 'ADP',
 'FI',
 'ANET',
 'BMY',
 'GILD',
 'SCHW',
 'DE',
 'TMUS',
 'ADI',
 'MDT',
 'VRTX',
 'MMC',
 'CB',
 'LMT',
 'KKR',
 'MU',
 'SBUX',
 'PLD',
 'GEV',
 'LRCX',
 'UPS',
 'NKE',
 'SO',
 'MO',
 'EQIX',
 'INTC',
 'PYPL',
 'KLAC',
 'ICE',
 'AMT',
 'ELV',
 'CME',
 'APH',
 'TT',
 'CRWD',
 'CMG',
 'DUK',
 'CDNS',
 'PH',
 'SHW',
 'MDLZ',
 'MSI',
 'CI',
 'AON'

In [35]:
tickers_short = tickers[:50]

In [36]:
tickers_short

['PARA', 'FOX', 'NWS']

In [ ]:
def fetch_dates(tickers):
    # Loop through each stock ticker
    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        # Fetch the earnings history (past earnings reports)
        earnings_history = ticker.earnings_dates
        
        # Skip if no earnings data is available
        if earnings_history is None:
            print(f"{ticker_symbol}: possibly delisted; no earnings dates found")
            continue
        
        # Filter the earnings history to include only non-NaN Reported EPS
        filtered_earnings = earnings_history[earnings_history['Reported EPS'].notna()]

        # Iterate through each earnings report and determine the correct date classification
        for index, row in filtered_earnings.iterrows():
            report_time = pd.to_datetime(row.name)  # Access the index (which is the earnings date) and convert it to datetime
            if report_time.hour >= 16:  # After 4 PM, classify as next day because the trade based on this knowledge can effectively only be executed the day after
                adjusted_date = (report_time + pd.Timedelta(days=1)).date()
            else:  # Otherwise, use the same day
                adjusted_date = report_time.date()

            # Add each adjusted earnings date as a separate row in the DataFrame
            new_row = {'Ticker': ticker_symbol, 'Date': adjusted_date}
            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

        time.sleep(8)

In [ ]:


# URL of the website containing the S&P 500 tickers
url = 'https://www.slickcharts.com/sp500'

# Fetch the page content with headers to avoid being blocked
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'class': 'table table-hover table-borderless table-sm'})
    tickers = []
    
    # Extract tickers from the table
    if table:
        for row in table.find('tbody').find_all('tr'):
            columns = row.find_all('td')
            ticker = columns[2].text.strip()  # The third column contains the ticker symbol
            tickers.append(ticker)
    else:
        print("Table not found in the page")
else:
    print("Failed to fetch S&P 500 tickers")
    tickers = []

# Display the tickers
print(tickers)


# Assume 'tickers' is already defined as the list of tickers

# Divide the tickers into smaller lists of 25 elements each
ticker_groups = [tickers[i:i + 25] for i in range(0, len(tickers), 25)]

# Display the ticker groups
for idx, group in enumerate(ticker_groups):
    print(f"Group {idx + 1}: {group}")



# Initialize an empty DataFrame to store valid earnings data
data = pd.DataFrame(columns=['Ticker', 'Date'])

# Iterate over all the ticker groups
for i in range(len(ticker_groups)):
    tickers = ticker_groups[i]
    print(f"Processing tickers from group {i + 1}: {tickers}")

    # You can add any additional processing logic here for each group

    # Loop through each stock ticker
    for ticker_symbol in tickers:
        ticker = yf.Ticker(ticker_symbol)
        # Fetch the earnings history (past earnings reports)
        earnings_history = ticker.earnings_dates
        
        # Skip if no earnings data is available
        if earnings_history is None:
            print(f"{ticker_symbol}: possibly delisted; no earnings dates found")
            continue
        
        # Filter the earnings history to include only non-NaN Reported EPS
        filtered_earnings = earnings_history[earnings_history['Reported EPS'].notna()]

        # Iterate through each earnings report and determine the correct date classification
        for index, row in filtered_earnings.iterrows():
            report_time = pd.to_datetime(row.name)  # Access the index (which is the earnings date) and convert it to datetime
            if report_time.hour >= 16:  # After 4 PM, classify as next day because the trade based on this knowledge can effectively only be executed the day after
                adjusted_date = (report_time + pd.Timedelta(days=1)).date()
            else:  # Otherwise, use the same day
                adjusted_date = report_time.date()

            # Add each adjusted earnings date as a separate row in the DataFrame
            new_row = {'Ticker': ticker_symbol, 'Date': adjusted_date}
            data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

        time.sleep(8)


    

# # Display the resulting DataFrame
# print(data)
# path_to_save = project_general_path / 'data' / 'sp500_earnings_dates.xlsx'
# data.to_excel(path_to_save, index=False)

['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'TSLA', 'GOOGL', 'AVGO', 'GOOG', 'BRK.B', 'JPM', 'LLY', 'V', 'XOM', 'UNH', 'MA', 'COST', 'PG', 'WMT', 'HD', 'NFLX', 'JNJ', 'CRM', 'ABBV', 'BAC', 'ORCL', 'MRK', 'KO', 'CVX', 'CSCO', 'WFC', 'ACN', 'NOW', 'MCD', 'PEP', 'IBM', 'AMD', 'DIS', 'LIN', 'TMO', 'ABT', 'ADBE', 'PM', 'ISRG', 'GE', 'GS', 'INTU', 'CAT', 'QCOM', 'TXN', 'VZ', 'BKNG', 'AXP', 'PLTR', 'T', 'SPGI', 'RTX', 'MS', 'BLK', 'PFE', 'NEE', 'HON', 'DHR', 'CMCSA', 'AMGN', 'PGR', 'LOW', 'AMAT', 'UNP', 'TJX', 'ETN', 'BA', 'BSX', 'C', 'UBER', 'SYK', 'COP', 'BX', 'PANW', 'ADP', 'FI', 'ANET', 'BMY', 'GILD', 'SCHW', 'DE', 'TMUS', 'ADI', 'MDT', 'VRTX', 'MMC', 'CB', 'LMT', 'KKR', 'MU', 'SBUX', 'PLD', 'GEV', 'LRCX', 'UPS', 'NKE', 'SO', 'MO', 'EQIX', 'INTC', 'PYPL', 'KLAC', 'ICE', 'AMT', 'ELV', 'CME', 'APH', 'TT', 'CRWD', 'CMG', 'DUK', 'CDNS', 'PH', 'SHW', 'MDLZ', 'MSI', 'CI', 'AON', 'APO', 'PNC', 'SNPS', 'USB', 'REGN', 'CL', 'WM', 'WELL', 'ZTS', 'MCO', 'MCK', 'TDG', 'CEG', 'EMR', 'MMM', 'ORLY', 'ITW',

BRK.B: $BRK.B: possibly delisted; no earnings dates found


BRK.B: possibly delisted; no earnings dates found
Processing tickers from group 2: ['ORCL', 'MRK', 'KO', 'CVX', 'CSCO', 'WFC', 'ACN', 'NOW', 'MCD', 'PEP', 'IBM', 'AMD', 'DIS', 'LIN', 'TMO', 'ABT', 'ADBE', 'PM', 'ISRG', 'GE', 'GS', 'INTU', 'CAT', 'QCOM', 'TXN']
Processing tickers from group 3: ['VZ', 'BKNG', 'AXP', 'PLTR', 'T', 'SPGI', 'RTX', 'MS', 'BLK', 'PFE', 'NEE', 'HON', 'DHR', 'CMCSA', 'AMGN', 'PGR', 'LOW', 'AMAT', 'UNP', 'TJX', 'ETN', 'BA', 'BSX', 'C', 'UBER']
Processing tickers from group 4: ['SYK', 'COP', 'BX', 'PANW', 'ADP', 'FI', 'ANET', 'BMY', 'GILD', 'SCHW', 'DE', 'TMUS', 'ADI', 'MDT', 'VRTX', 'MMC', 'CB', 'LMT', 'KKR', 'MU', 'SBUX', 'PLD', 'GEV', 'LRCX', 'UPS']
Processing tickers from group 5: ['NKE', 'SO', 'MO', 'EQIX', 'INTC', 'PYPL', 'KLAC', 'ICE', 'AMT', 'ELV', 'CME', 'APH', 'TT', 'CRWD', 'CMG', 'DUK', 'CDNS', 'PH', 'SHW', 'MDLZ', 'MSI', 'CI', 'AON', 'APO', 'PNC']


Failed to get ticker 'APO' reason: Expecting value: line 1 column 1 (char 0)


Processing tickers from group 6: ['SNPS', 'USB', 'REGN', 'CL', 'WM', 'WELL', 'ZTS', 'MCO', 'MCK', 'TDG', 'CEG', 'EMR', 'MMM', 'ORLY', 'ITW', 'COF', 'GD', 'EOG', 'WMB', 'MAR', 'BDX', 'APD', 'CTAS', 'NOC', 'ADSK']
Processing tickers from group 7: ['AJG', 'CSX', 'FTNT', 'HLT', 'TGT', 'FDX', 'ECL', 'OKE', 'RCL', 'ABNB', 'TFC', 'GM', 'CARR', 'WDAY', 'BK', 'FCX', 'ROP', 'DLR', 'CVS', 'HCA', 'SRE', 'PCAR', 'AZO', 'TRV', 'NXPI']
Processing tickers from group 8: ['JCI', 'NSC', 'SLB', 'SPG', 'KMI', 'AMP', 'AFL', 'ALL', 'CPRT', 'FICO', 'AEP', 'ROST', 'PWR', 'CMI', 'GWW', 'VST', 'MET', 'MSCI', 'URI', 'PSA', 'O', 'PSX', 'AXON', 'AIG', 'D']
Processing tickers from group 9: ['HWM', 'PAYX', 'EW', 'KMB', 'FIS', 'DFS', 'PCG', 'NEM', 'LULU', 'TEL', 'MPC', 'FAST', 'PEG', 'PRU', 'AME', 'KVUE', 'RSG', 'DHI', 'KR', 'LHX', 'COR', 'BKR', 'CCI', 'DAL', 'CBRE']
Processing tickers from group 10: ['CTSH', 'CTVA', 'VRSK', 'TRGP', 'XEL', 'A', 'F', 'SYY', 'IT', 'VLO', 'EXC', 'YUM', 'OTIS', 'IR', 'GLW', 'DELL', 'MNST'

BF.B: $BF.B: possibly delisted; no earnings dates found


BF.B: possibly delisted; no earnings dates found
Processing tickers from group 21: ['PARA', 'FOX', 'NWS']


In [11]:
data

,Ticker,Date
0,AAPL,2024-10-31
1,AAPL,2024-08-01
2,AAPL,2024-05-02
3,AAPL,2024-02-01
4,AAPL,2023-11-02
...,...,...
3928,PARA,2024-02-28
3929,PARA,2023-11-02
3930,PARA,2023-08-07
3931,PARA,2023-05-04


In [20]:
path_to_save = project_general_path / 'data' / 'sp500_earnings_dates.xlsx'
data.to_excel(path_to_save, index=False)